In [ ]:
#Andrew Lonie ArcPy Lab 1 

#Total Coding Time Taken: 22+ Hours

# First Deliverable:

#making a file (geojson) for WA State counties

# Setup
from subprocess import call
import os
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy
from arcpy import env
env.overwriteOutput = True
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"

#Step1:
# Getting a list of all the unique county FIPS codes together
#worked with Jeremy here..helped me get the appending right
   
def createcounties(mypath, myfile, mytable):
    
    env.workspace = mypath	
    
    cursor1 = arcpy.da.SearchCursor(myfile, ["COUNTYFP10"])
    
    countyfips = []
    
    for row1 in cursor1:
        if row1[0] not in countyfips:
            countyfips.append(row1[0])
    del row1
    del cursor1
    
    ["001", etc]
    print countyfips
    
# here I worked with getting all of the geojson files together
# This was extremely difficult for me 
# I didn't know multiple cusors could exist on one code
    
    for fipsnum in countyfips:
        cursor2 = arcpy.da.SearchCursor(mytable, ["CountyName", "FIPSCounty"])
        county_name = ""
        for row2 in cursor2:
            if fipsnum == row2[1]:
                county_name = row2[0]
        del row2
        del cursor2
        county_file_name = county_name + ".shp"
        county_geo = county_name + ".geojson"
        county_geo_path = mypath + "\\" + county_geo
        county_file_path = mypath + "\\" + county_file_name
        
        print county_file_name
        
        arcpy.CreateFeatureclass_management(mypath, county_file_name, "POLYGON", myfile, "", "", myfile)
    
        cursor1 = arcpy.da.SearchCursor(myfile, ["SHAPE@", "COUNTYFP10", "POP2013"])
        cursor2 = arcpy.da.InsertCursor(county_file_name, ["SHAPE@", "COUNTYFP10", "POP2013"])
        
        for row1 in cursor1:
            print row1[3]
            print fipsnum
            if row1[1] == fipsnum:
                cursor2.insertRow(row1)
        del row1
        del cursor1
        del cursor2
        
      
        # Where it says "'-s_srs','EPSG:2927'", that's what the "saep_bg10.shp" projection is
        # 'EPSG:2927'is the call comm	and from 'handy link' link on canvas
        # Using the imported ogr2ogr lines at the top we converted a .shp to a .geojson without actually overridding the file.
        # Jeremy helped me again here, I struggled finding out how to get the file from overwritting my .shp with .geojson      
	call(['C:\\OSGeo4W\\bin\\ogr2ogr',	
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',
      county_geo_path,
      county_file_path])

	#List of WA State counties.
    
1) 	King County Population: 	1980645
2) 	Pierce County Population: 	812568
3) 	Snohomish County Population: 	729395
4) 	Spokane County Population: 	479204
5) 	Clark County Population: 	433535
6) 	Thurston County Population: 	259293
7) 	Kitsap County Population: 	252586
8) 	Yakima County Population: 	246176
9) 	Whatcom County Population: 	204582
10) Benton County Population: 	181005
        
             


createcounties("C:\\Users\\alonie\\Desktop\\temp", "saep_bg10.shp", "WashingtonFIPS.dbf")  